In [1]:
import pandas as pd

PATH = "../data"
DATASET = 'news_29_80k'

data = pd.read_csv(f"{PATH}/dataset/{DATASET}.csv").dropna()



In [2]:
data

,id,channel,text,genre,word_count,sign_count,smile_count
0,6649,akashevarova,вагнер может вернуться на фронт именно как ваг...,1,208.0,0.0,21.0
1,6648,akashevarova,киев заявляет о взятии клещеевки что под бахму...,1,225.0,0.0,7.0
2,6647,akashevarova,миграционное право самая отвратительная сфера ...,1,196.0,3.0,3.0
3,6646,akashevarova,узнала что после череды прилетов по москвасити...,1,81.0,0.0,2.0
4,6645,akashevarova,официальные власти ничего не говорят но вот пе...,1,113.0,0.0,2.0
...,...,...,...,...,...,...,...
67961,4824,trends,лайфхак о котором вы даже не догадывались паре...,9,25.0,0.0,0.0
67962,4823,trends,некоторым кто мечтал озвучивать сериалы очень ...,9,53.0,0.0,4.0
67963,4822,trends,джеймсу кэмерону предложили снять сериал о про...,9,38.0,0.0,1.0
67964,4816,trends,bloodborne портируют в minecraft энтузиаст под...,9,34.0,0.0,2.0


In [3]:
from transformers import RobertaTokenizerFast
from transformers import AutoTokenizer, AutoModel

tokenizer = AutoTokenizer.from_pretrained("cointegrated/rubert-tiny2", max_len=512)
model = AutoModel.from_pretrained("cointegrated/rubert-tiny2", num_labels=29)

c:\Working directory\ml-projects\aiNews\.venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


OSError: We couldn't connect to 'https://huggingface.co' to load this file, couldn't find it in the cached files and it looks like cointegrated/rubert-tiny2 is not the path to a directory containing a file named config.json.
Checkout your internet connection or see how to run the library in offline mode at 'https://huggingface.co/docs/transformers/installation#offline-mode'.

In [ ]:
import torch


def embed_bert_cls(text, model, tokenizer):
    t = tokenizer(text, padding=True, truncation=True, return_tensors='pt')
    with torch.no_grad():
        model_output = model(**{k: v.to(model.device) for k, v in t.items()})
    embeddings = model_output.last_hidden_state[:, 0, :]
    embeddings = torch.nn.functional.normalize(embeddings)
    return embeddings[0].cpu().numpy()

print(embed_bert_cls('И снова даров', model, tokenizer).shape)

In [ ]:
print(embed_bert_cls(data["text"].iloc[0], model, tokenizer))

In [ ]:
import numpy as np
data['emb'] = ''

In [ ]:
data['emb'] = data["text"].apply(lambda x: embed_bert_cls(x, model, tokenizer))
data

In [ ]:
data.to_pickle("../data/dataset/news_29_80k_clean_with_features_embs.pkl")